# **Import Tools**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.initializers import Constant
import re
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
import pickle
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
import string
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from transformers import pipeline
import numpy as np
import random
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from keras.layers import (LSTM, Embedding, BatchNormalization, Dense, TimeDistributed, Dropout, Bidirectional,Flatten, GlobalMaxPool1D)
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, SpatialDropout1D, Conv1D, LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from spacy.util import compounding
from spacy.util import minibatch
from collections import defaultdict
from collections import Counter
from sklearn.metrics import (precision_score, recall_score, f1_score, classification_report, accuracy_score, confusion_matrix)
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
import random
import keras

# **Read Data**

In [ ]:
mul_em = pd.read_csv('/kaggle/input/multiclass-emotion-model-dataset/data_train.csv')

emotion_dataset = pd.read_csv('/kaggle/input/emotion-dataset/training.csv')
t1 = {0:'sadness', 1:'joy', 2:'love', 3:'anger', 4:'fear', 5:'surprise'}
emotion_dataset['emotion'] = emotion_dataset['label'].map(t1)
new_col1 = ['Emotion','Text']
c1 = emotion_dataset[['emotion','text']]
c1.columns = new_col1
emotion_dataset = c1


edos = pd.read_csv('/kaggle/input/edos-1m-data-sentiment-analysis/EDOS 1M.csv')
column1 = ['Emotion','Text']
s_c1 = edos[['eb+_emot','uttr']]
s_c1.columns = column1
edos = s_c1

isear = pd.read_csv('/kaggle/input/isear-dataset/eng_dataset.csv')
column3 = ['Emotion', 'Text']
s_c3 = isear[['sentiment', 'content']]
s_c3.columns = column3
isear = s_c3

data = pd.concat([emotion_dataset,mul_em,edos,isear], ignore_index=True)
data.info()

# Fixed Max Sequence Length

In [ ]:
m_len = 60
data = data[(data['Text'].str.len() <= m_len)]
data.reset_index(drop=True, inplace=True)
data.info()

# Drop Duplicates

In [ ]:
data = data.drop_duplicates(subset='Text', keep='first')
data.reset_index(drop=True, inplace=True)
data.info()

# Value Count

In [ ]:
u_count = data['Emotion'].nunique()
u_count

In [ ]:
data_c = data['Emotion'].value_counts()
print(data_c)

# Select Emotions

In [ ]:
t_cou = {
    'joyful': 10000,
    'sad': 10000,
    'hopeful': 10000,
    'afraid': 10000,
    'caring': 10000,
    'guilty': 7613,
    'annoyed': 10000,
    'surprised': 10000,
}

n_data = []

for emotion, count in t_cou.items():
    fil_rows = data[data['Emotion'] == emotion].head(count)
    n_data.append(fil_rows)

data = pd.concat(n_data, ignore_index=True)
data = data.sample(frac=1).reset_index(drop=True)


In [ ]:
data.info()

In [ ]:
data_c = data['Emotion'].value_counts()
print(data_c)

# **Text Cleaning**

In [ ]:
def c_corpus(x):
    x = str(x).lower()
    x = re.sub('\[.*?\]', '', x)
    x = re.sub('\n', '', x)
    x = re.sub('\w*\d\w*', '', x)
    x = re.sub('<.*?>+', '', x)
    return x

data['Text'] = data['Text'].apply(c_corpus)
data.head()

# **LabelEncoder**

In [ ]:
c = { 'joyful': 0,'sad': 1,'hopeful': 2,'afraid': 3,'caring': 4,'guilty': 5,'annoyed': 6,'surprised': 7}

data['Emotion'] = data['Emotion'].map(c)

In [ ]:
data.head()

In [ ]:
a = LabelEncoder()
a.fit(data['Emotion'])
data['Emotion'] = a.transform(data['Emotion'])

# **Split Data**

In [ ]:
x1 = data['Text']
y1 = data['Emotion']
x1_train, x1_test, y1_train, y1_test = train_test_split(x1, y1, random_state=42)

# **TfidfVectorizer**

In [ ]:
vec = TfidfVectorizer()
vec.fit(x1_train)
x1_train = vec.transform(x1_train)
x1_test  = vec.transform(x1_test)

# **MultinomialNB**

In [ ]:
MNB = MultinomialNB()
MNB.fit(x1_train, y1_train)
y_p = MNB.predict(x1_test)
acc = accuracy_score(y1_test, y_p)
print(f"Test accuracy: {acc:.4f}")

In [ ]:
print(classification_report(y1_test, y_p))

In [ ]:
plt.figure(figsize=(7,5))
c_mnb = confusion_matrix(y1_test, y_p)
sns.heatmap(c_mnb, linewidths=0.01, annot=True, fmt='.1f', cmap='Blues', cbar=False)
plt.xlabel('Predicted Labels', fontsize=12)
plt.ylabel('True Labels', fontsize=12)
plt.title('Confusion Matrix', fontsize=16)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()

In [ ]:
with open('updated_mnb_model.pkl', 'wb') as m_fi:
    pickle.dump(MNB, m_fi)

In [ ]:
with open('updated_mnb_model.pkl', 'rb') as m_fi:
    mnb = pickle.load(m_fi)

# **SGDClassifier**

In [ ]:
"""
p_grid = {
    'loss': ['hinge', 'log', 'modified_huber'],
    'penalty': ['l1', 'l2', 'elasticnet'],
    'alpha': [0.0001, 0.001, 0.01],
    'max_iter': [1000, 2000, 3000]
}

g_search = GridSearchCV(sgd, pa_grid, cv=5)
print("Best Hyperparameters: ", g_search.best_params_)
print("Best Score: ", g_search.best_score_)

"""
s_gd = SGDClassifier()
s_gd.fit(x1_train, y1_train)
pre = s_gd.predict(x1_test)
acc = accuracy_score(y1_test, pre)*100
print(f"Test accuracy of Stochastic Gradient Descent: {acc:.4f}")

In [ ]:
print(classification_report(y1_test, pre))

In [ ]:
plt.figure(figsize=(7,5))
c_sgd = confusion_matrix(y1_test, pre)
sns.heatmap(c_sgd, linewidths=0.01, annot=True, fmt='.1f', cmap='Blues', cbar=False)
plt.xlabel('Predicted Labels', fontsize=12)
plt.ylabel('True Labels', fontsize=12)
plt.title('Confusion Matrix', fontsize=16)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()

In [ ]:
with open('updated_sgd_model.pkl', 'wb') as m_fi:
    pickle.dump(s_gd, m_fi)

In [ ]:
with open('updated_mnb_model.pkl', 'rb') as m_fi:
    sgd = pickle.load(m_fi)

# **KNeighborsClassifier**

In [ ]:
kn = KNeighborsClassifier()
kn_model = kn.fit(x1_train, y1_train)
kn_pre = kn_model.predict(x1_test)

In [ ]:
kn_score = round(accuracy_score(y1_test, kn_pre) * 100, 2)
print("The accuracy score of KNN is = ",kn_score)

In [ ]:
print(classification_report(y1_test, kn_pre))

In [ ]:
plt.figure(figsize=(7,5))
c_knn = confusion_matrix(y1_test, kn_pre)
sns.heatmap(c_knn, linewidths=0.01, annot=True, fmt='.1f', cmap='Blues', cbar=False)
plt.xlabel('Predicted Labels', fontsize=12)
plt.ylabel('True Labels', fontsize=12)
plt.title('Confusion Matrix', fontsize=16)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()

In [ ]:
with open('updated_kn_model.pkl', 'wb') as m_fi:
    pickle.dump(kn_model, m_fi)

In [ ]:
with open('updated_kn_model.pkl', 'rb') as m_fi:
    knc = pickle.load(m_fi)

# **RandomForestClassifier**

In [ ]:
rdf = RandomForestClassifier()
r_model = rdf.fit(x1_train, y1_train)
r_pre = r_model.predict(x1_test)
r_score = round(accuracy_score(y1_test, r_pre) * 100, 2)
print("The accuracy score of the model is = ",r_score)

In [ ]:
print(classification_report(y1_test, r_pre))

In [ ]:
plt.figure(figsize=(7,5))
c_random = confusion_matrix(y1_test, r_pre)
sns.heatmap(c_random, linewidths=0.01, annot=True, fmt='.1f', cmap='Blues', cbar=False)
plt.xlabel('Predicted Labels', fontsize=12)
plt.ylabel('True Labels', fontsize=12)
plt.title('Confusion Matrix', fontsize=16)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()

In [ ]:
with open('updated_rdf_model.pkl', 'wb') as m_fi:
    pickle.dump(r_model, m_fi)

In [ ]:
with open('updated_rdf_model.pkl', 'rb') as m_fi:
    rdf = pickle.load(m_fi)

# **LinearSVC**

In [ ]:
s_vc = LinearSVC()
s_model = s_vc.fit(x1_train, y1_train)
s_pre = s_model.predict(x1_test)
s_score = round(accuracy_score(y1_test, s_pre) * 100, 2)
print("The accuracy score of LinearSVC is = ",s_score)

In [ ]:
print(classification_report(y1_test, s_pre))

In [ ]:
plt.figure(figsize=(7,5))
c_svc = confusion_matrix(y1_test, s_pre)
sns.heatmap(c_svc, linewidths=0.01, annot=True, fmt='.1f', cmap='Blues', cbar=False)
plt.xlabel('Predicted Labels', fontsize=12)
plt.ylabel('True Labels', fontsize=12)
plt.title('Confusion Matrix', fontsize=16)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()

In [ ]:
with open('updated_svc_model.pkl', 'wb') as m_fi:
    pickle.dump(s_model, m_fi)

In [ ]:
with open('updated_svc_model.pkl', 'rb') as m_fi:
    svc = pickle.load(m_fi)

# Pre-trained Model

In [ ]:
! pip -q install transformers

In [ ]:
c = { 'joyful': 0,'sad': 1,'hopeful': 2,'afraid': 3,'caring': 4,'guilty': 5,'annoyed': 12,'surprised': 14}

data['Emotion'] = data['Emotion'].map(c)

In [ ]:
pre_model = pipeline( model="facebook/bart-large-mnli",framework="pt",)
user_input = "hi how are you "
r = pre_model(
    user_input,
    candidate_labels = ['joyful','sad','hopeful','afraid','caring','guilty','annoyed','surprised'],)

# LSTM

In [ ]:
c = { 'joyful': 0,'sad': 1,'hopeful': 2,'afraid': 3,'caring': 4,'guilty': 5,'annoyed': 6,'surprised': 7}

data['Emotion'] = data['Emotion'].map(c)

In [ ]:
data.info()

In [ ]:
data.head()

In [ ]:
x2 = data['Text']
y2 = data['Emotion']
x_train, x_test, y_train, y_test = train_test_split(x2, y2, random_state=42)
t = Tokenizer()
t.fit_on_texts(x_train)
w_index = t.word_index
voc_size = len(t.word_index) + 1
x_train = pad_sequences(t.texts_to_sequences(x_train),maxlen = 60)
x_test = pad_sequences(t.texts_to_sequences(x_test),maxlen = 60)

### Glove word embedding

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

In [ ]:
d = {}
with open('glove.6B.300d.txt','r') as fi:
    for i in fi:
        v1 = i.split()
        w = v1[0]
        wr = np.asarray(v1[1:],'float32')
        d[w]=wr

    
e_mat = np.zeros((voc_size, 300))
for i, j in w_index.items():
    e_vec = d.get(i)
    if e_vec is not None:
        e_mat[j] = e_vec

### Training

In [ ]:
num_emotions = 8
i_shape = (60,)
input_s = Input(shape = i_shape, dtype='int32')
layer_em = tf.keras.layers.Embedding(voc_size, 300, weights=[e_mat], input_length=60, trainable=False)
sequences_em = layer_em(input_s)
l = SpatialDropout1D(0.2)(sequences_em)
l = Conv1D(64, 5, activation='relu')(l)
l = Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2))(l)
l = Dense(512, activation='relu')(l)
l = Dropout(0.2)(l)
l = Dense(512, activation='relu')(l)
o = Dense(num_emotions, activation='softmax')(l)
lstm_model = tf.keras.Model(input_s, o)
lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
y_train_en = to_categorical(y_train, num_classes=num_emotions)
y_test_en = to_categorical(y_test, num_classes=num_emotions)

In [ ]:
history = lstm_model.fit(x_train, y_train_en, batch_size = 256, epochs = 30, validation_data = (x_test, y_test_en))

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

### Evaluation

In [ ]:
y_test_encoded = to_categorical(y_test, num_classes=num_emotions)
ac = lstm_model.evaluate(x_test, y_test_encoded)

In [ ]:
lstm_model.save("updated_bilstm_model_for_8_categories.h5")

In [ ]:
l_model = load_model("/kaggle/input/updated-bilstm-with-hc/updated_bilstm_model_for_8_categories.h5")

In [ ]:
a = { 0:'joyful',1:'sad',2:'hopeful',3:'afraid',4:'caring',5:'guilty',6:'annoyed',7:'surprised'}
i = ['We wll both grow during this time, and who knows what the future holds? ']
x1 = pad_sequences(t.texts_to_sequences(i),maxlen = 60)
s = l_model.predict(x1, verbose=0)
p_i = np.argmax(s)
p_l = a[p_i] 
p_score = s[0, p_i]

print(f"Predicted Class: {p_l}")
